In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
import os
import datetime
import IPython
import IPython.display

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import pandas as pd
from pandas import read_csv
import numpy as np
import random
from pandas import DataFrame
from numpy import concatenate
from pandas import concat
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score 


import matplotlib.patches as mpatches
import matplotlib.pylab as plt
import matplotlib as mpl
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 7)
plt.style.use('fast')
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
from sklearn.neural_network import MLPRegressor


np.random.seed(123) # for reproducibility
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from scikeras.wrappers import KerasClassifier, KerasRegressor
import keras_tuner

In [2]:
# CARGAR LA BASE DE DATOS
from pandas import read_csv
datos = read_csv('Barrancabermeja_Lags.csv', encoding='latin-1', sep=";")
# Dropping the id and date columns
datos = datos.drop(['Municipio', 'Año', 'Semana_Epi', 'Temp_Seca_Max_Dia_Prom_1',
       'Temp_Seca_Max_Dia_Prom_2','Temp_Seca_Max_Dia_Prom_3','Temp_Seca_Max_Dia_Prom_4', 
       'Temp_Seca_Max_Dia_Prom_5','Temp_Seca_Max_Dia_Prom_6', 'Temp_Seca_Max_Sem_1',
       'Temp_Seca_Max_Sem_2', 'Temp_Seca_Max_Sem_3', 'Temp_Seca_Max_Sem_4',
       'Temp_Seca_Max_Sem_5', 'Temp_Seca_Max_Sem_6','Temp_Seca_Min_Dia_Prom_1',
       'Temp_Seca_Min_Dia_Prom_2','Temp_Seca_Min_Dia_Prom_3', 'Temp_Seca_Min_Dia_Prom_4',
       'Temp_Seca_Min_Dia_Prom_5', 'Temp_Seca_Min_Dia_Prom_6','Temp_Seca_Min_Sem_1',
       'Temp_Seca_Min_Sem_2','Temp_Seca_Min_Sem_3', 'Temp_Seca_Min_Sem_4', 
       'Temp_Seca_Min_Sem_5','Temp_Seca_Min_Sem_6','Prom_Dia_Preci_1',
       'Prom_Dia_Preci_2', 'Prom_Dia_Preci_3', 'Prom_Dia_Preci_4',
       'Prom_Dia_Preci_5', 'Prom_Dia_Preci_6','Total_Preci_Sem_1',
       'Total_Preci_Sem_2', 'Total_Preci_Sem_3','Total_Preci_Sem_4',
       'Total_Preci_Sem_5', 'Total_Preci_Sem_6','Temp_Max_Dia_Prom_1',
       'Temp_Max_Dia_Prom_2','Temp_Max_Dia_Prom_3', 'Temp_Max_Dia_Prom_4',
       'Temp_Max_Dia_Prom_5','Temp_Max_Dia_Prom_6','Temp_Max_Sem_1','Temp_Max_Sem_2',
       'Temp_Max_Sem_3', 'Temp_Max_Sem_4', 'Temp_Max_Sem_5','Temp_Max_Sem_6',
       'Temp_Min_Dia_Prom_1','Temp_Min_Dia_Prom_2', 'Temp_Min_Dia_Prom_3',
       'Temp_Min_Dia_Prom_4','Temp_Min_Dia_Prom_5', 'Temp_Min_Dia_Prom_6',
       'Temp_Min_Sem_1', 'Temp_Min_Sem_2', 'Temp_Min_Sem_3', 'Temp_Min_Sem_4',
       'Temp_Min_Sem_5', 'Temp_Min_Sem_6','Hum_Rel_Max_1','Hum_Rel_Max_2',
       'Hum_Rel_Max_3', 'Hum_Rel_Max_4', 'Hum_Rel_Max_5','Hum_Rel_Max_6',
       'Hum_Rel_Min_1', 'Hum_Rel_Min_2','Hum_Rel_Min_3', 'Hum_Rel_Min_4',
       'Hum_Rel_Min_5', 'Hum_Rel_Min_6'],axis=1)
datos.columns

Index(['Casos_Dengue', 'Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Sem',
       'Temp_Seca_Min_Dia_Prom', 'Temp_Seca_Min_Sem', 'Prom_Dia_Preci',
       'Total_Preci_Sem', 'Temp_Max_Dia_Prom', 'Temp_Max_Sem',
       'Temp_Min_Dia_Prom', 'Temp_Min_Sem', 'Hum_Rel_Max', 'Hum_Rel_Min'],
      dtype='object')

In [3]:
from sklearn.impute import KNNImputer
knnImputer = KNNImputer (missing_values=np.nan, n_neighbors=2, 
                         weights="uniform", metric="nan_euclidean")
# Ajustamos el modelo e imputamos los missing values
knnImputer.fit(datos[['Hum_Rel_Max']])
datos['Hum_Rel_Max'] = knnImputer.transform(datos[['Hum_Rel_Max']]).ravel()

knnImputer.fit(datos[['Hum_Rel_Min']])
datos['Hum_Rel_Min'] = knnImputer.transform(datos[['Hum_Rel_Min']]).ravel()

datos.shape

(624, 13)

In [4]:
train_size = int(len(datos) * 0.7)
test_size = len(datos) - train_size
train, test = datos.iloc[0:train_size], datos.iloc[train_size:len(datos)]
print(len(train), len(test))

436 188


In [5]:
f_transformer = MinMaxScaler(feature_range=(0, 1))
f_columns = ['Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Sem', 'Temp_Seca_Min_Dia_Prom', 'Temp_Seca_Min_Sem', 'Prom_Dia_Preci',
             'Total_Preci_Sem', 'Temp_Max_Dia_Prom', 'Temp_Max_Sem',
             'Temp_Min_Dia_Prom', 'Temp_Min_Sem', 'Hum_Rel_Max', 'Hum_Rel_Min']

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
train.head()

,Casos_Dengue,Temp_Seca_Max_Dia_Prom,Temp_Seca_Max_Sem,Temp_Seca_Min_Dia_Prom,Temp_Seca_Min_Sem,Prom_Dia_Preci,Total_Preci_Sem,Temp_Max_Dia_Prom,Temp_Max_Sem,Temp_Min_Dia_Prom,Temp_Min_Sem,Hum_Rel_Max,Hum_Rel_Min
0,1,0.701493,0.565217,0.702632,0.863636,0.018991,0.018993,0.611940,0.515152,0.232558,0.272727,0.476190,0.439024
1,4,0.417910,0.565217,0.481579,0.500000,0.069424,0.069340,0.283582,0.454545,0.258398,0.318182,0.555556,0.682927
2,1,0.597015,0.521739,0.518421,0.727273,0.037139,0.037082,0.462687,0.393939,0.387597,0.545455,0.634921,0.585366
3,1,0.597015,0.565217,0.510526,0.681818,0.000000,0.000000,0.462687,0.454545,0.387597,0.454545,0.634921,0.536585
4,1,0.686567,0.565217,0.542105,0.727273,0.000000,0.000000,0.582090,0.545455,0.077519,0.272727,0.555556,0.707317


In [6]:
train.describe()

,Casos_Dengue,Temp_Seca_Max_Dia_Prom,Temp_Seca_Max_Sem,Temp_Seca_Min_Dia_Prom,Temp_Seca_Min_Sem,Prom_Dia_Preci,Total_Preci_Sem,Temp_Max_Dia_Prom,Temp_Max_Sem,Temp_Min_Dia_Prom,Temp_Min_Sem,Hum_Rel_Max,Hum_Rel_Min
count,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000,436.000000
mean,13.238532,0.503112,0.528171,0.464685,0.470392,0.174922,0.175022,0.418106,0.501633,0.391325,0.454962,0.623290,0.526072
std,13.904204,0.160478,0.105136,0.186300,0.183003,0.168981,0.168952,0.154346,0.144669,0.193129,0.175761,0.125147,0.155258
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.402985,0.456522,0.337500,0.318182,0.039091,0.039117,0.314552,0.393939,0.258398,0.318182,0.555556,0.439024
50%,7.000000,0.522388,0.543478,0.473684,0.454545,0.130091,0.130540,0.418657,0.515152,0.361757,0.420455,0.634921,0.512195
75%,20.000000,0.611940,0.586957,0.580921,0.590909,0.266565,0.266581,0.522388,0.606061,0.516796,0.545455,0.634921,0.609756
max,60.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
dengue_transformer = MinMaxScaler(feature_range=(0, 1))
dengue_transformer = dengue_transformer.fit(train[['Casos_Dengue']])
train['Casos_Dengue'] = dengue_transformer.transform(train[['Casos_Dengue']])
train['Casos_Dengue'].head()

0    0.016667
1    0.066667
2    0.016667
3    0.016667
4    0.016667
Name: Casos_Dengue, dtype: float64

In [8]:
test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
test.head()

,Casos_Dengue,Temp_Seca_Max_Dia_Prom,Temp_Seca_Max_Sem,Temp_Seca_Min_Dia_Prom,Temp_Seca_Min_Sem,Prom_Dia_Preci,Total_Preci_Sem,Temp_Max_Dia_Prom,Temp_Max_Sem,Temp_Min_Dia_Prom,Temp_Min_Sem,Hum_Rel_Max,Hum_Rel_Min
436,4,0.597015,0.695652,0.763158,0.545455,0.110994,0.110944,0.552239,0.666667,0.749354,0.727273,0.634921,0.512195
437,2,0.313433,0.434783,0.552632,0.454545,0.212281,0.212240,0.238806,0.393939,0.516796,0.545455,0.634921,0.585366
438,3,0.283582,0.347826,0.684211,0.681818,0.071534,0.071450,0.223881,0.393939,0.516796,0.681818,0.555556,0.487805
439,5,0.567164,0.543478,0.947368,0.818182,0.182739,0.182695,0.477612,0.545455,0.387597,0.500000,0.555556,0.609756
440,4,0.492537,0.521739,0.710526,0.681818,0.071112,0.071149,0.477612,0.545455,0.413437,0.363636,0.634921,0.414634


In [9]:
test['Casos_Dengue'] = dengue_transformer.transform(test[['Casos_Dengue']])
test['Casos_Dengue'].head()

436    0.066667
437    0.033333
438    0.050000
439    0.083333
440    0.066667
Name: Casos_Dengue, dtype: float64

In [10]:
def create_dataset(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [11]:
time_steps = 10
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train, train.Casos_Dengue, time_steps)
X_test, y_test = create_dataset(test, test.Casos_Dengue, time_steps)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape )

(426, 10, 13) (426,) (178, 10, 13) (178,)


In [12]:
from tensorflow.keras.layers import InputLayer

def RNN_model(neuronas,
               dropout_rate,
               num_layers,
               activation = 'relu',
               init_mode='uniform',
              ):
    
    model = keras.Sequential()
    
    # CAPA DE ENTRADA
    model.add(keras.layers.InputLayer(input_shape=(X_train.shape[1], X_train.shape[2])))
    
    
    #CAPAS INTERMEDIAS
    for i in range(num_layers):
        model.add(keras.layers.SimpleRNN(units=neuronas,
                                    activation=activation,
                                    return_sequences = True,
                                    dropout=dropout_rate,
                                    kernel_initializer=init_mode))
        
        
    #CAPA DE SALIDA
    model.add(keras.layers.Dense(1, activation="linear"))
    
    #COMPILACIÓN DEL MODELO
    model.compile(optimizer='adam', loss = 'mse', metrics = ['mse'])
    
    return model

In [13]:
#HIPERPARÁMETROS

neuronas = [50, 70, 100]
activation = ['sigmoid','relu', 'tanh']
init_mode = ['uniform', 'normal', 'zero']
num_layers = [3, 5, 7]
dropout_rate = [0.1, 0.3, 0.5]

#optimizer = ['adam', 'sgd', 'rmsprop']
#batch_size = [40, 60, 80]
#epochs = [10, 50, 100]
#momentum = [0.2, 0.4, 0.5]

param_grid = dict(model__neuronas=neuronas,
                  model__activation=activation,
                  model__init_mode=init_mode,
                  model__dropout_rate=dropout_rate,
                  model__num_layers=num_layers)
                  #batch_size=batch_size,
                  #epochs=epochs,
                  #optimizer__momentum=momentum

In [14]:
model =  KerasRegressor(model=RNN_model, verbose=0)

rs = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring='neg_mean_squared_error',
                        n_jobs=-1, cv=5, n_iter=20, verbose=0,random_state = 123, return_train_score = True)

In [15]:
fitted_model = rs.fit(X_train, y_train)

In [16]:
resultados = pd.DataFrame(fitted_model.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)\
    .head(10)

,param_model__num_layers,param_model__neuronas,param_model__init_mode,param_model__dropout_rate,param_model__activation,mean_test_score,std_test_score,mean_train_score,std_train_score
0,3,100,uniform,0.5,tanh,NaN,NaN,NaN,NaN
1,5,50,zero,0.1,sigmoid,NaN,NaN,NaN,NaN
2,3,100,normal,0.5,relu,NaN,NaN,NaN,NaN
3,5,50,uniform,0.3,tanh,NaN,NaN,NaN,NaN
4,5,70,uniform,0.3,sigmoid,NaN,NaN,NaN,NaN
5,5,50,uniform,0.1,tanh,NaN,NaN,NaN,NaN
6,7,70,uniform,0.3,tanh,NaN,NaN,NaN,NaN
7,7,100,zero,0.5,sigmoid,NaN,NaN,NaN,NaN
8,5,50,normal,0.1,relu,NaN,NaN,NaN,NaN
9,3,100,zero,0.1,sigmoid,NaN,NaN,NaN,NaN


In [17]:
print("Best: %f using %s" % (fitted_model.best_score_, fitted_model.best_params_))

Best: nan using {'model__num_layers': 3, 'model__neuronas': 100, 'model__init_mode': 'uniform', 'model__dropout_rate': 0.5, 'model__activation': 'tanh'}
